In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove6b100dtxt/glove.6B.100d.txt
/kaggle/input/burnout-tweets-analysis/burnout.csv
/kaggle/input/burnout-tweets-analysis/__results__.html
/kaggle/input/burnout-tweets-analysis/df.csv
/kaggle/input/burnout-tweets-analysis/__notebook__.ipynb
/kaggle/input/burnout-tweets-analysis/__output__.json
/kaggle/input/burnout-tweets-analysis/custom.css
/kaggle/input/burnout-tweets-analysis/__results___files/__results___60_0.png
/kaggle/input/burnout-tweets-analysis/__results___files/__results___58_0.png
/kaggle/input/burnout-tweets-analysis/__results___files/__results___62_0.png
/kaggle/input/burnout-tweets-analysis/__results___files/__results___50_0.png
/kaggle/input/glove840b300dtxt/glove.840B.300d.txt


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
plt.style.use('ggplot')
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from collections import  Counter
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score,
    confusion_matrix
)

In [3]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC

from keras.layers.recurrent import GRU
from keras.layers.core import Activation


from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.metrics import make_scorer

## Recalculating CountVecotrizer Vector representation of Tweet Text

In [4]:
df = pd.read_csv('../input/burnout-tweets-analysis/df.csv')

In [5]:
#Split data into training, validation and test sets
y=df['burnout']

xtrain, xtest, ytrain, ytest = train_test_split(df['Tweet Text'].values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [6]:
print (xtrain.shape)
print (xtest.shape)

(1503,)
(376,)


In [7]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')



# Fitting Count Vectorizer to training set
ctv.fit(xtrain)
xtrain_ctv =  ctv.transform(xtrain) 
xtest_ctv = ctv.transform(xtest)

In [8]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(xtrain)
xtrain_tfv =  tfv.transform(xtrain) 
xtest_tfv = tfv.transform(xtest)

As a reminder, for our initial model pass, XGBoost using CountVectorizer's vector ranked the highest in model performance. 

Now, we will try and extend model performance by trying out some more methods to optimize our model's burnout predictions. 

Thank you: https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

## Grid Search

From skikit-learn: 
Grid Search is an exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a “fit” and a “score” method. It also implements “score_samples”, “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated grid-search over a parameter grid.

In this section, I'll talk about grid search using logistic regression.

In [9]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

Grid of parameters:

In [10]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

So, for SVD we evaluate 120 and 180 components and for logistic regression we evaluate three different values of C with l1 and l2 penalty. We can now start grid search on these parameters.

In [11]:
#Defining scoring metric
f1 = make_scorer(f1_score , pos_label=0, average='binary') #making sure f1 score is catching minority class (non-burnout class)

In [12]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=f1,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

In [13]:
# Fit Grid Search Model using Count Vect
model.fit(xtrain_ctv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:   12.9s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   14.6s finished


Best score: 0.539
Best parameters set:
	lr__C: 10
	lr__penalty: 'l2'
	svd__n_components: 180


The score is still the second to the worst of minority class' f1 scores from all prior training runs.

Let's see what TFIDF produces.

In [14]:
# Fit Grid Search Model using TFIDF
model.fit(xtrain_tfv, ytrain)  
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0877s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  14 out of  24 | elapsed:    0.6s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.9s finished


Best score: 0.560
Best parameters set:
	lr__C: 10
	lr__penalty: 'l2'
	svd__n_components: 180


TFIDF actually did a little better. So let's move onto grid search on higher performing models - such as NB.

In [15]:
# Grid Search on TFIDF

nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=f1,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: 0.583
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0113s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0113s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished


Great! This is a better result  than what we got with Log Reg. Let us try Grid Search using Count Vect.

In [16]:
# Grid Search on Count Vect

nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=f1,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_ctv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: 0.610
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0149s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished


Much better! (Though same result prior to grid search. So grid search can be advantageous at times)

## Word Vectors

Another way to improve our model is to work on the actual data itself. 

* Count Vectorizer just counts occurrences of words which puts bias on most frequent words and loses rare words. Though data processing is efficient. 

* Tf-Idf helps with bias against most frequent words. Weights are put on each word. There is penalization of frequent words. 

* Word2Vec treats each word in corpus like an atomic entity and generates a vector for each word. In this sense Word2vec is very much like Glove - both treat words as the smallest unit to train on. 

* Fasttext (which is essentially an extension of word2vec model), treats each word as composed of character ngrams. So the vector for a word is made of the sum of this character n grams. For example the word vector “apple” is a sum of the vectors of the n-grams “<ap”, “app”, ”appl”, ”apple”, ”apple>”, “ppl”, “pple”, ”pple>”, “ple”, ”ple>”, ”le>” (assuming hyperparameters for smallest ngram[minn] is 3 and largest ngram[maxn] is 6)

In [17]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [05:34, 6573.44it/s]

Found 2196017 word vectors.


In [18]:
# this function creates a normalized vector for the whole sentence

def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [19]:
# create sentence vectors using the above function for training and test set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xtest_glove = [sent2vec(x) for x in tqdm(xtest)]

100%|██████████| 376/376 [00:00<00:00, 2061.50it/s]


In [20]:
xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)

Let's get back to XGBoost and fit our new GloVe embeddings on it.

In [21]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict(xtest_glove)

print (classification_report(ytest, predictions))

[19:33:45] WARNING: ../src/learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


              precision    recall  f1-score   support

           0       0.70      0.57      0.63       136
           1       0.78      0.86      0.82       240

    accuracy                           0.76       376
   macro avg       0.74      0.72      0.73       376
weighted avg       0.75      0.76      0.75       376



In [22]:
confusion_matrix(ytest, predictions)

array([[ 78,  58],
       [ 33, 207]])

Awesome! Though confusion matrix shows that GloVe embeddings had more incorrectly classified tweets (false positives and false negatives), Recall and f1 for minority class was highest so far. This is promising. It shows that embeddings are superior than simple word count/frequency vectors. Next up is Deep Learning since this is the modern way of text classification.